In [5]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.optim as optim

In [6]:
df_fa = pd.read_csv('/home/syb/Code/coupling/python/file/data/SDI_sub/sc_mode_1.csv')
df_md = pd.read_csv('/home/syb/Code/coupling/python/file/data/SDI_sub/sc_mode_2.csv')
df_num = pd.read_csv('/home/syb/Code/coupling/python/file/data/SDI_sub/sc_mode_3.csv')
df_label = pd.read_csv('/home/syb/Code/coupling/python/file/data/sc_mode_label.csv')
df_info = pd.read_csv('/home/syb/Code/coupling/python/file/data/sc_mode_info.csv')
df_mode= [df_fa,df_md,df_num]

In [7]:

# feature_range控制压缩数据范围，默认[0,1]
scaler = MinMaxScaler(feature_range=[0,1]) # 实例化，调整0,1的数值可以改变归一化范围
 
X = scaler.fit_transform(df_fa.values)  # 将标签归一化到0,1之间
Y = scaler.fit_transform(df_info['mmse'].values)  # 将特征归于化到0,1之间
 
x = scaler.inverse_transform(X) # 将数据恢复至归一化之前

ValueError: Expected 2D array, got 1D array instead:
array=[28. 28. 29. 29. 29. 30. 29. 27. 28. 29. 29. 29. 27. 24. 28. 28. 28. 30.
 26. 30. 30. 30. 30. 26. 29. 28. 30. 30. 30. 29. 29. 27. 30. 28. 28. 26.
 29. 29. 27. 30. 30. 29. 29. 25. 27. 30. 27. 30. 23. 21. 26. 28. 27. 29.
 28. 29. 26. 29. 22. 25. 27. 28. 29. 29. 23. 25. 24. 29. 27. 29. 23. 27.
 24. 24. 27. 26. 22. 18. 19. 23. 15. 23.  5. 23. 12. 13. 24. 15. 22. 25.
 17. 22. 16. 21. 27. 26. 23. 19. 25.  5. 18. 22.  7.  7. 15. 17. 20. 19.
 13.  6. 18.  9. 11.  3. 13. 13. 20. 23. 23. 29. 30. 29. 30. 29. 30. 29.
 28. 29. 30. 29. 26. 28. 27. 28. 30. 29. 30. 29. 28. 30. 27. 25. 26. 28.
 28. 30. 22. 28. 26. 29. 26. 27. 24. 28. 29. 28. 28. 26. 29. 27. 28. 27.
 25. 25. 21. 17. 19. 22. 22. 18. 12. 23. 16. 15.  6. 20. 20. 26. 13. 22.
 26. 19. 22. 16. 20. 22. 18. 29. 28. 27. 30. 30. 30. 30. 29. 29. 27. 30.
 30. 29. 28. 29. 28. 30. 27. 27. 30. 27. 29. 28. 29. 26. 28. 30. 23. 23.
 23. 23. 25. 27. 24. 22. 28. 26. 21. 24. 27. 27. 26. 27. 28. 29. 

In [3]:

 
 
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_output, n_neuron1, n_neuron2,n_layer):  # n_feature为特征数目，这个数字不能随便取,n_output为特征对应的输出数目，也不能随便取
        self.n_feature=n_feature
        self.n_output=n_output
        self.n_neuron1=n_neuron1
        self.n_neuron2=n_neuron2
        self.n_layer=n_layer
        super(Net, self).__init__()
        self.input_layer = torch.nn.Linear(self.n_feature, self.n_neuron1) # 输入层
        self.hidden1 = torch.nn.Linear(self.n_neuron1, self.n_neuron2) # 1类隐藏层    
        self.hidden2 = torch.nn.Linear(self.n_neuron2, self.n_neuron2) # 2类隐藏
        self.predict = torch.nn.Linear(self.n_neuron2, self.n_output) # 输出层
 
    def forward(self, x):
        '''定义前向传递过程'''
        out = self.input_layer(x)
        out = torch.relu(out) # 使用relu函数非线性激活
        out = self.hidden1(out)
        out = torch.relu(out)
        for i in range(self.n_layer-1):
            out = self.hidden2(out)
            out = torch.relu(out) 

        out = self.hidden2(out)
        out = self.predict( # 回归问题最后一层不需要激活函数
            out
        )  # 除去feature_number与out_prediction不能随便取，隐藏层数与其他神经元数目均可以适当调整以得到最佳预测效果
        return out

In [ ]:

'''训练部分'''

def MTrain(train_data):
    feature_number = 13  # 设置特征数目
    out_prediction = 1  # 设置输出数目
    learning_rate = 0.01  # 设置学习率
    epochs = 50  # 设置训练代数
    net = Net(n_feature=feature_number,
                        n_output=out_prediction,
                        n_layer=1,
                        n_neuron1=20,
                        n_neuron2=20) # 这里直接确定了隐藏层数目以及神经元数目，实际操作中需要遍历
    optimizer = optim.Adam(net.parameters(), learning_rate)  # 使用Adam算法更新参数
    criteon = torch.nn.MSELoss()  # 误差计算公式，回归问题采用均方误差
    
    for epoch in range(epochs):  # 整个数据集迭代次数
        net.train() # 启动训练模式
        for batch_idx, (data, target) in enumerate(train_data):
            logits = net.forward(data)  # 前向计算结果（预测结果）
            loss = criteon(logits, target)  # 计算损失
            optimizer.zero_grad()  # 梯度清零
            loss.backward()  # 后向传递过程
            optimizer.step()  # 优化权重与偏差矩阵
    
        logit = []  # 这个是验证集，可以根据验证集的结果进行调参，这里根据验证集的结果选取最优的神经网络层数与神经元数目
        target = []
        net.eval() # 启动测试模式
        for data, targets in validation:  # 输出验证集的平均误差
            logits = net.forward(data).detach().numpy()
            targets=targets.detach().numpy()
            target.append(targets[0])
            logit.append(logits[0])
        average_loss =  criteon(torch.tensor(logit), torch.tensor(target))


In [ ]:
r2_re = []
y_pred_3 = []
y_true_3 = []

r_list = []
p_list = []
mode = 0

#计算三种模态各个的分类性能
for mode in range(1):
    y_pred_mode = []
    y_true_mode = []
    #fig, ax = plt.subplots(figsize=(3, 3))
    #把fa和info合并成一个表格
    df_sc1= pd.concat([df_mode[mode],df_info],join="outer",axis=1)
    for i in range(7):
        center = i+1
        df = df_sc1.sample(frac=1).reset_index(drop=True)
        #df = df[df['label'] !=2]
        #df = df['label'].replace(3,0)
        
        #df.loc[df['label']==3,'label'] = 0
        df_test = df[df['center']== center].drop(axis=1,columns=['center','label'])
        df_train = df[df['center'] != center].drop(axis=1,columns=['center','label'])
        
        y_test = df_test['mmse'].values
        y_train = df_train['mmse'].values
        x_test = df_test.drop(axis=1,columns=['mmse']).values
        x_train = df_train.drop(axis=1,columns=['mmse']).values

        
      
        y_pred = model.predict(x_test)

        y_pred_mode.extend(y_pred)
        y_true_mode.extend(y_test)
        r2_re.append(pearsonr(y_pred,y_test)[0])

    y_pred_3.append(y_pred_mode)
    y_true_3.append(y_true_mode)
    r_list.append(pearsonr(y_pred_mode,y_true_mode)[0])
    p_list.append(pearsonr(y_pred_mode,y_true_mode)[0])

    fig, ax = plt.subplots(figsize=(8, 8))

    df_y = pd.DataFrame( data= np.transpose([y_pred_mode,y_true_mode]), columns=['pred','true'])
    

    sns.regplot(data= df_y, x='true',y= 'pred')
    
    modelName = str(model)

    #plt.scatter(y_true_mode,y_pred_mode)

    rp = pearsonr(y_pred_mode,y_true_mode)
    
    
    plt.xlim([0,35])
    plt.ylim([0,35])
    
    plt.title(modelName+', r= '+ str(round(rp[0],5)) +', p = '+ str(rp[1]))
    plt.show()